Import the main libraries for this project

In [176]:
import pandas as pd
import numpy as np
from datetime import datetime

Load the datasets to analize them separately

In [177]:
# Let's make a preliminary analysis to the datasets
races_df = pd.read_csv("./dataset/races.csv")
cyclist_df = pd.read_csv("./dataset/cyclists.csv")

**Ciclist analysis**
The key for cyclist table is column **_url** that is the name of the cyclist, while in races table the column **cyclist** is a foreign key

In [196]:
def check_key(df, column_name):
    # Check if there are no NaN values and if all values are unique
    return df[column_name].notna().all() and df[column_name].is_unique
def check_in(df1, df2, col1, col2):
    return df1[col1].isin(df2[col2]).all()

print(check_key(cyclist_df, '_url')) # This column is a key for clyclist dataset
print(check_in(races_df, cyclist_df, 'cyclist', '_url')) # These columns can be used for join

True
True


Some columns have missing values in the cyclist table. For birth_year and nationality we can try to fix them, while we cannot fix weight and height columns since they are NaN for half of the table.

In [180]:
cyclist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   float64
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: float64(3), object(3)
memory usage: 287.7+ KB


Only cyclist **scott-davies** has missing nationality. We can fix its nationality and birth_year by looking on the web.

In [182]:
 # Only one cyclist has nan nationality.
print(cyclist_df.loc[cyclist_df.nationality[cyclist_df.nationality.isna()].index])
def fix_cyclist(df):
    mask = df['nationality'].isna()
    cyclists = set(df.loc[mask, '_url'])
    for c in cyclists:
        mask = df["_url"] == c
        df.loc[mask, 'nationality'] = 'Britain'
        df.loc[mask, 'birth_year'] = 1995

# Fix using Wikipedia
fix_cyclist(cyclist_df)
print(cyclist_df.loc[cyclist_df.nationality[cyclist_df._url == 'scott-davies'].index])

           _url           name  birth_year  weight  height nationality
9  scott-davies  Scott  Davies         NaN     NaN     NaN         NaN


These cyclist instead have missing birth_year, they are few we can try to fix them.

In [ ]:
cyclist_df.loc[cyclist_df.birth_year[cyclist_df.birth_year.isna()].index]

**Races analysis**
Some columns have missing values in races table (**points**, **uci_points**, **climb_total**, **profile**, **average_temperature**, **cyclist_age**, **cycilist_team**).

We can try to fix columns **points**, **cyclist_age** since there are few missing values.
We can drop the column **average_temperature** since there are too many missing values.
We can try to analyze the distribution of **climb_total** over **profile** to try to esitimate and fix the columns.
We can assume that missing values if **cyclist_team** implies 'No team'

In [181]:
races_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist              589865 non-null  object 
 12  cyclist_age          589752 non-null  float64
 13  is_tarmac            589865 non-null  bool   
 14  is_cobbled           589865 non-null  bool   
 15  is_gravel        

There is some noise in the date we can remove the hours, minutes and seconds of start of the races

In [197]:
# Let's remove the noise from dates of races
def reformat_dates(df):
    df['date'] = pd.to_datetime(df['date'])
    return df['date'].dt.date

races_df['date'] = reformat_dates(races_df)
print(races_df['date'].unique())


[datetime.date(1978, 7, 5) datetime.date(2016, 9, 3)
 datetime.date(2019, 7, 28) ... datetime.date(1976, 3, 9)
 datetime.date(2016, 3, 27) datetime.date(2010, 5, 8)]


We can try to check if **points** column is mutually eclusive with **uci_points** column. They aren't

In [186]:
# Points and UCI_Points are not mutually exclusive columns
def are_xor_columns(df, col1, col2):
    mask = df[col1].isna() ^ df[col2].isna()
    res = df.loc[mask, 'cyclist'].tolist()
    return len(res) == len(df)

print(are_xor_columns(races_df, 'points', 'uci_points'))

False


**is_cobbled** and **is_gravel** are all false (we can drop them).
**is_tarmac** is sometimes true and false. It's ok but when it's false also the other columns are false (Noise!!). 

In [198]:
# Some columns are all False
print(races_df.is_cobbled.unique())
print(races_df.is_gravel.unique())
print(races_df.is_tarmac.unique())

[False]
[False]
[ True False]


In [ ]:
The delta column have some negative values and also the delta is 0 for many positions different from 0 (Very noisy column!!)

In [188]:
#A column has inconsistent values. (Negative delta)
print(sorted(races_df.delta.unique())[:100])
# Replace negative values with nan
races_df['delta'] = races_df['delta'].mask(races_df['delta'] < 0, np.nan)

[-6906.0, -5562.0, -2937.0, -2638.0, -2635.0, -2574.0, -2567.0, -2564.0, -2560.0, -2550.0, -2546.0, -2545.0, -2542.0, -2541.0, -2514.0, -2513.0, -2510.0, -2509.0, -2505.0, -2504.0, -2500.0, -2487.0, -2486.0, -2485.0, -2482.0, -2481.0, -2479.0, -2477.0, -2475.0, -2473.0, -2469.0, -2457.0, -2448.0, -2446.0, -2444.0, -2440.0, -2438.0, -2437.0, -2434.0, -2432.0, -2430.0, -2428.0, -2425.0, -2424.0, -2419.0, -2416.0, -2413.0, -2410.0, -2408.0, -2397.0, -2394.0, -2393.0, -2391.0, -2385.0, -2384.0, -2378.0, -2367.0, -2365.0, -2359.0, -2358.0, -2356.0, -2355.0, -2353.0, -2352.0, -2349.0, -2348.0, -2346.0, -2345.0, -106.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0]


In [ ]:
First position in a race is 0. We can check it looking on the web.

In [189]:
# First position in a race is 0. Delta is always 0 for position 0 and different from 0 for position. Checked using Internet
delta0 = ((races_df['position'] == 0) & (races_df['delta'] == 0)).sum()
delta1 = ((races_df['position'] == 1) & (races_df['delta'] == 0)).sum()
print(delta0)
print(delta1)

# Much Much noise in delta column. All positions have some 0 deltas
positions = sorted(races_df.loc[races_df['delta'].isna(), 'position'].unique())
print(positions)
positions = sorted(races_df.loc[races_df['delta'] == 0, 'position'].unique())
print(positions)

5281
2738
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 104]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 1

In [ ]:
Some races have missing profiles. Try to fix looking to the climb_total distribution

In [190]:
# Profile is numerical (difficulty of trace) missing for some races
print(races_df.profile.unique())

[ 1.  5. nan  3.  2.  4.]


The startlist are integers representing how strong the lineup is. They are sparse.

In [199]:
races_df.startlist_quality.unique()[:100]

array([1241,  821, 1699,  804, 1551,  899,  659,  388,  900,  541,  830,
        789,  602,  817, 1400, 1161, 1040,  896,  791,  819,  670,  225,
        520, 1057,  809,  828,  722,  747,  714,  815,  376,  621,  760,
        798,  933, 1994, 1437, 1362,  884, 1150,  971,  881, 1112, 1175,
        891,  878,  400,  936,  692,  727, 1002,  928, 1196, 1489,  687,
        585,  835,  673, 1328,  885,  502,  861,  982,  923, 1036, 1690,
        925,  989, 1109, 1084,  803,  792,  548,  668, 1713, 1520, 1959,
        883,  859, 2047, 1024,  533, 1269,  751, 1202,  570, 1703, 1416,
        251, 1158, 1139,  657, 1048,  521, 1034, 1099,  880, 1029,  627,
        340], dtype=int64)

Now we can build the joined table and analyze the data together

In [192]:
class DataCleaner:
    def __init__(self, races_csv, cyclist_csv, merged_csv=None):
        # So keep ref to all datasets and make one joined
        self.races_df = pd.read_csv(races_csv, parse_dates=["date"])
        self.cyclist_df = pd.read_csv(cyclist_csv)

        if merged_csv == None:
            self.df = pd.merge(
                self.races_df, self.cyclist_df, left_on="cyclist", right_on="_url", how="inner"
            )
        else:
            self.df = pd.read_csv(merged_csv, parse_dates=["date"])

        # Delete useless columns
        self.delete_column("_url_y")
        self.delete_column("name_y")

        # Delete columns that are all false
        self.delete_column("is_cobbled")
        self.delete_column("is_gravel")

        # Rename to understand better
        self.df.rename(
            columns={
                "name_x": "location",
                "_url_x": "url",
            },
            inplace=True,
        )

        dm.export_csv() # Save the merged version of dataset  
    def delete_column(self, col):
        self.df.drop(columns=[col], inplace=True)
    def fill_nan(self, column, value):
        self.df[column].fillna(value, inplace=True)
    def export_csv(self):
        return self.df.to_csv("./dataset/dataset.csv")

In [193]:
dm = DataCleaner("./dataset/races.csv", "./dataset/cyclists.csv") # Initialize the datasets

In [194]:
dm.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   url                  589865 non-null  object        
 1   location             589865 non-null  object        
 2   points               589388 non-null  float64       
 3   uci_points           251086 non-null  float64       
 4   length               589865 non-null  float64       
 5   climb_total          442820 non-null  float64       
 6   profile              441671 non-null  float64       
 7   startlist_quality    589865 non-null  int64         
 8   average_temperature  29933 non-null   float64       
 9   date                 589865 non-null  datetime64[ns]
 10  position             589865 non-null  int64         
 11  cyclist              589865 non-null  object        
 12  cyclist_age          589752 non-null  float64       
 13  is_tarmac     

In [195]:
# Try to fix missing birth_year or age using the other column. Nothing they always nan at the same time

ages = dm.df.groupby('cyclist')['cyclist_age'].unique()

ages = ages.apply(lambda x: np.isnan(x).all())
# names for which is true
ages = ages[ages == True].index

for idx, name in enumerate(ages):
    print(idx,name)
    print(cyclist_df.loc[cyclist_df._url == name].birth_year)

0 alexandr-osipov
2536   NaN
Name: birth_year, dtype: float64
1 antonio-zanini
894   NaN
Name: birth_year, dtype: float64
2 batik-odriozola
6080   NaN
Name: birth_year, dtype: float64
3 carlos-garcia
2515   NaN
Name: birth_year, dtype: float64
4 filippo-simonetti
2408   NaN
Name: birth_year, dtype: float64
5 javier-luquin
4384   NaN
Name: birth_year, dtype: float64
6 nevens-guy
3551   NaN
Name: birth_year, dtype: float64
7 nicolai-kosyakov
3046   NaN
Name: birth_year, dtype: float64
8 oscar-pumar
4142   NaN
Name: birth_year, dtype: float64
9 scott-davies
9    1995.0
Name: birth_year, dtype: float64
10 sergei-jermachenko
6072   NaN
Name: birth_year, dtype: float64
11 thierry-lauder
4756   NaN
Name: birth_year, dtype: float64
12 vladimir-malakov
601   NaN
Name: birth_year, dtype: float64
